In [9]:
# Notes on modified stuff:
    # changed argsparse stuff. cuz not terminal running this shit 
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D
from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os
import sys

from PIL import Image

In [21]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", type=str, default = 'Batches/Batch_1',
    help="path to input dataset")
ap.add_argument("-e", "--epochs", type=int, default=25,
    help="# of epochs to train our network for")
ap.add_argument("-p", "--plot", type=str, default="plot.png",
    help="path to output loss/accuracy plot")

#--------------------------------------------------------------------------------------------------------------------
# New stuff
from easydict import EasyDict
args = args = EasyDict({
    "dataset": 'Batches/testing',
    "epochs": 5,
    'plot':'plot.png'
})
#---------------------------------------------------------------------------------------------------------------------

In [37]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images(args["dataset"]))
data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
    # load the image, swap color channels, and resize it to be a fixed
    # 128x128 pixels while ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (200, 200))
    print(np.shape(image))
    #sys.exit(0)
    # update the data and labels lists, respectively
    data.append(image)
    print(np.shape(data))
    sys.exit(0)
    labels.append(label)


[INFO] loading images...
(200, 200, 3)
(1, 200, 200, 3)


SystemExit: 0

In [38]:
np.shape(data)


(1, 200, 200, 3)

In [30]:
# convert the data and labels to NumPy arrays
data = np.array(data)
#------------------------------------------------------------------------------------------------
labelss = np.array(labels)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labelss = lb.fit_transform(labelss)
#labels = np_utils.to_categorical(labels)
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labelss,
	test_size=0.25, stratify=labels, random_state=42)

# Prev errors:-------------------------------------------------------------------------------------------------------------
    # Found array with dim 3. Estimator expected <= 2, change labels -> labelss, otherwise run prev cell too

In [31]:
trainX.shape

(2250, 200, 200, 3)

In [26]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")
# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [27]:
# load VGG16, ensuring the head FC layer sets are left off, while at
# the same time adjusting the size of the input image tensor to the
# network
baseModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(200, 200, 3)))
# show a summary of the base model
print("[INFO] summary for base model...")
print(baseModel.summary())

[INFO] summary for base model...
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)  

In [28]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(3, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False
    
#------------------------
# changed dense(2... to dense(3... cuz 3 classes

In [39]:
# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = Adam(lr=1e-4)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size= 100),#32),
	steps_per_epoch= 1, #len(trainX) // 32,
	validation_data=valAug.flow(testX, testY),
	validation_steps= 1, #len(testX) // 32,
	epochs=args["epochs"])
#------------------------------------------------------------------------


[INFO] compiling model...
[INFO] training head...
Epoch 1/5
1/1 [==============================] - 86s 86s/step - loss: 2.2275 - accuracy: 0.5567 - val_loss: 1.2643 - val_accuracy: 0.5938
Epoch 2/5
1/1 [==============================] - 40s 40s/step - loss: 2.4444 - accuracy: 0.5667 - val_loss: 1.3848 - val_accuracy: 0.5625
Epoch 3/5
1/1 [==============================] - 25s 25s/step - loss: 2.1926 - accuracy: 0.5167 - val_loss: 1.5862 - val_accuracy: 0.6458
Epoch 4/5
1/1 [==============================] - 21s 21s/step - loss: 2.4707 - accuracy: 0.5733 - val_loss: 1.3179 - val_accuracy: 0.5625
Epoch 5/5
1/1 [==============================] - 20s 20s/step - loss: 1.8376 - accuracy: 0.5933 - val_loss: 1.4649 - val_accuracy: 0.4792


In [ ]:
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))
# plot the training loss and accuracy
N = args["epochs"]
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper right")
plt.savefig(args["plot"])